In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys.symbolic import PartiallySolvedSystem, ScaledSys, symmetricsys, get_logexp
from chempy import ReactionSystem, Substance
from chempy.kinetics.ode import get_odesys
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string("""
A -> B; 0.04
B + C -> A + C; 1e4
2 B -> B + C; 3e7
""", substance_factory=lambda formula: Substance(formula, composition={1: 1}))

In [ ]:
rsys.composition_balance_vectors()

In [ ]:
orisys, extra = get_odesys(rsys, SymbolicSys=ScaledSys, dep_scaling=1e6, description='original')

In [ ]:
orisys.linear_invariants, orisys.names, orisys.latex_names

In [ ]:
psysA = PartiallySolvedSystem.from_linear_invariants(orisys, ['A'], description='A')
psysB = PartiallySolvedSystem.from_linear_invariants(orisys, ['B'], description='B')
psysC = PartiallySolvedSystem.from_linear_invariants(orisys, ['C'], description='C')

In [ ]:
psysA.dep, psysB.dep, psysC.dep

In [ ]:
linsystems = [orisys, psysA, psysB, psysC]

def integrate_systems(systems, **kwargs):
    return [odesys.integrate(1e14, {'A': 1, 'B': 0, 'C': 0}, integrator='cvode',
                             record_rhs_tvals=True, record_jac_tvals=True, record_order=True,
                             atol=1e-6, rtol=1e-6, **kwargs) for odesys in systems]

In [ ]:
linresults = integrate_systems(linsystems, nsteps=5000)

In [ ]:
def plot_result(odesys, res, ax=None, trnsfm=lambda x: x):
    plot_keys = ['steps', 'rhs_tvals', 'jac_tvals']
    bgcolor = ['orange', 'darkblue', 'darkgreen']
    for idx, key in enumerate(plot_keys):
        for x in res.xout if key == 'steps' else trnsfm(res.info[key]):
            ax.axvline(x, idx/len(plot_keys), (idx+1)/len(plot_keys), c=bgcolor[idx], alpha=0.1)
    _ = res.plot(xscale='log', yscale='log', ax=ax)
    ax.legend(loc='best')
    ax.set_title(odesys.description +
                 (' (%d steps, %d rhs evals.)' % (res.info['n_steps'], res.info['nfev'])) +
                '' if res.info['success'] else ' - failed!')

In [ ]:
def plot_results(systems, results):
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    for idx, (odesys, res) in enumerate(zip(systems, results)):
        plot_result(odesys, res, ax=axes.flat[idx])

In [ ]:
plot_results(linsystems, linresults)

In [ ]:
LogLogSys = symmetricsys(get_logexp(1, 1e-20), get_logexp(1, 1e-12), exprs_process_cb=lambda exprs: [
                         sp.powsimp(expr.expand(), force=True) for expr in exprs])
logsystems = [LogLogSys.from_other(ls) for ls in linsystems]

In [ ]:
logresults = integrate_systems(logsystems, nsteps=15000, return_on_error=True)

In [ ]:
plot_results(logsystems, logresults)